# Data Wrangling with TableOps

Durbyn.jl includes a lightweight, dplyr-inspired data manipulation module called `TableOps`. Built on top of Julia's [Tables.jl](https://github.com/JuliaData/Tables.jl) interface, it provides familiar verbs for reshaping, filtering, grouping, and summarising tabular data — everything you need to prepare time series data for modelling.

This chapter teaches TableOps using the **M3 Competition** dataset, a benchmark collection of 3,003 real-world time series.

## Loading Data

### From CSV

In [1]:
using Durbyn
using Durbyn.TableOps
using CSV
using Tables

# Load M3 monthly data
data_dir = joinpath(@__DIR__, "..", "data")
tbl_monthly = Tables.columntable(CSV.File(joinpath(data_dir, "M3_MONTHLY.csv")))

┌ Info: Durbyn.jl is under active development.
│ API may change without notice. Bugs and performance issues may exist.
│ Please report issues at: https://github.com/taf-society/Durbyn.jl/issues
└ @ Durbyn /home/akayr/Development/Open Source/Julia/Durbyn.jl/src/Durbyn.jl:90


(data = String3["M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3"  …  "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3"], series = String7["N1402", "N1402", "N1402", "N1402", "N1402", "N1402", "N1402", "N1402", "N1402", "N1402"  …  "N2829", "N2829", "N2829", "N2829", "N2829", "N2829", "N2829", "N2829", "N2829", "N2829"], period = String7["MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY"  …  "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY"], type = String15["MICRO", "MICRO", "MICRO", "MICRO", "MICRO", "MICRO", "MICRO", "MICRO", "MICRO", "MICRO"  …  "OTHER", "OTHER", "OTHER", "OTHER", "OTHER", "OTHER", "OTHER", "OTHER", "OTHER", "OTHER"], description = ["SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", 


The `CSV.File` returns a Tables.jl-compatible source; `Tables.columntable` converts it to a `NamedTuple` of vectors — the format TableOps expects.

### Understanding the Data Structure

In [2]:
# Quick overview of the table
glimpse(tbl_monthly)

Table glimpse
  Rows: 167562
  Columns: 9
  data                :: String3  [String3("M3"), String3("M3"), String3("M3"), String3("M3"), String3("M3"), …]
  series              :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  period              :: String7  [String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]
  description         :: String  ["SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", …]
  value_type          :: String3  [String3("x"), String3("x"), String3("x"), String3("x"), String3("x"), …]
  time                :: Float64  [1990.0, 1990.08333333333, 1990.16666666667, 1990.25, 1990.33333333333, …]
  date                :: Dates.Date  [Dates.Da


The M3 dataset has the following columns:

| Column | Type | Description |
|--------|------|-------------|
| `data` | String | Dataset identifier (`"M3"`) |
| `series` | String | Unique series identifier (e.g., `"N1402"`) |
| `period` | String | Frequency (`"MONTHLY"`, `"QUARTERLY"`, `"YEARLY"`) |
| `type` | String | Domain (e.g., `"MICRO"`, `"MACRO"`, `"FINANCE"`) |
| `description` | String | Human-readable series description |
| `value_type` | String | `"x"` = training data, `"xx"` = test (hold-out) data |
| `time` | Float64 | Decimal time index |
| `date` | String/Date | Calendar date |
| `value` | Float64 | Observed value |

## Core Verbs

### `select` — Choose Columns

In [ ]:
# Select specific columns
subset = select(tbl_monthly, :series, :date, :value, :value_type)
glimpse(subset)

# Drop columns by selecting everything else
core = select(tbl_monthly, :series, :value_type, :date, :value)

Table glimpse
  Rows: 167562
  Columns: 4
  series              :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  date                :: Dates.Date  [Dates.Date("1990-01-01"), Dates.Date("1990-02-01"), Dates.Date("1990-03-01"), Dates.Date("1990-04-01"), Dates.Date("1990-05-01"), …]
  value               :: Float64  [2640.0, 2640.0, 2160.0, 4200.0, 3360.0, …]
  value_type          :: String3  [String3("x"), String3("x"), String3("x"), String3("x"), String3("x"), …]


(series = String7["N1402", "N1402", "N1402", "N1402", "N1402", "N1402", "N1402", "N1402", "N1402", "N1402"  …  "N2829", "N2829", "N2829", "N2829", "N2829", "N2829", "N2829", "N2829", "N2829", "N2829"], value_type = String3["x", "x", "x", "x", "x", "x", "x", "x", "x", "x"  …  "xx", "xx", "xx", "xx", "xx", "xx", "xx", "xx", "xx", "xx"], date = [Dates.Date("1990-01-01"), Dates.Date("1990-02-01"), Dates.Date("1990-03-01"), Dates.Date("1990-04-01"), Dates.Date("1990-05-01"), Dates.Date("1990-06-01"), Dates.Date("1990-07-01"), Dates.Date("1990-08-01"), Dates.Date("1990-09-01"), Dates.Date("1990-10-01")  …  Dates.Date("0006-02-01"), Dates.Date("0006-03-01"), Dates.Date("0006-04-01"), Dates.Date("0006-05-01"), Dates.Date("0006-06-01"), Dates.Date("0006-07-01"), Dates.Date("0006-08-01"), Dates.Date("0006-09-01"), Dates.Date("0006-10-01"), Dates.Date("0006-11-01")], value = [2640.0, 2640.0, 2160.0, 4200.0, 3360.0, 2400.0, 3600.0, 1920.0, 4200.0, 4560.0  …  1361.1, 1331.4, 1316.5, 1316.5, 1286.8,


### `query` — Filter Rows

`query` is the equivalent of `dplyr::filter`. It takes a predicate function that receives each row as a `NamedTuple`.

In [4]:
# Keep only training data
trn = query(tbl_monthly, row -> row.value_type == "x")
glimpse(trn)

# Keep only test data
test = query(tbl_monthly, row -> row.value_type == "xx")

# Filter by series type
macro_series = query(tbl_monthly, row -> row.type == "MACRO" && row.value_type == "x")

# Filter by date range
recent = query(tbl_monthly,
    row -> row.value_type == "x" && row.time >= 1990.0)

# Combine conditions
subset = query(tbl_monthly,
    row -> row.type == "FINANCE" &&
           row.value_type == "x" &&
           row.value > 1000)

Table glimpse
  Rows: 141858
  Columns: 9
  data                :: String3  [String3("M3"), String3("M3"), String3("M3"), String3("M3"), String3("M3"), …]
  series              :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  period              :: String7  [String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]
  description         :: String  ["SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", …]
  value_type          :: String3  [String3("x"), String3("x"), String3("x"), String3("x"), String3("x"), …]
  time                :: Float64  [1990.0, 1990.08333333333, 1990.16666666667, 1990.25, 1990.33333333333, …]
  date                :: Dates.Date  [Dates.Da

(data = String3["M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3"  …  "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3"], series = String7["N2522", "N2522", "N2522", "N2522", "N2522", "N2522", "N2522", "N2522", "N2522", "N2522"  …  "N2666", "N2666", "N2666", "N2666", "N2666", "N2666", "N2666", "N2666", "N2666", "N2666"], period = String7["MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY"  …  "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY", "MONTHLY"], type = String15["FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE"  …  "FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE", "FINANCE"], description = ["Bankers' dollar acceptances outstanding, end of period", "Bankers' dollar acceptances outstanding, end of period", "Bankers' dollar acceptances outstanding, end o

> ⚠️ **Warning**: Handling Missing Values in `query`
>
>
> If a column might contain `missing`, comparisons return `missing` (not `true`/`false`), which causes an error. Use `coalesce`:
>

In [5]:
temp = query(tbl_monthly, row -> coalesce(row.value > 100, false))
glimpse(temp)

Table glimpse
  Rows: 167557
  Columns: 9
  data                :: String3  [String3("M3"), String3("M3"), String3("M3"), String3("M3"), String3("M3"), …]
  series              :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  period              :: String7  [String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]
  description         :: String  ["SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", …]
  value_type          :: String3  [String3("x"), String3("x"), String3("x"), String3("x"), String3("x"), …]
  time                :: Float64  [1990.0, 1990.08333333333, 1990.16666666667, 1990.25, 1990.33333333333, …]
  date                :: Dates.Date  [Dates.Da


### `arrange` — Sort Rows

In [6]:
# Sort by series and then by time
sorted = arrange(tbl_monthly, :series, :time)

# Sort with custom ordering (descending)
# First arrange by series, then by value descending
by_value = arrange(tbl_monthly, :series)  # then use query for ordering within
glimpse(by_value)

Table glimpse
  Rows: 167562
  Columns: 9
  data                :: String3  [String3("M3"), String3("M3"), String3("M3"), String3("M3"), String3("M3"), …]
  series              :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  period              :: String7  [String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]
  description         :: String  ["SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", …]
  value_type          :: String3  [String3("x"), String3("x"), String3("x"), String3("x"), String3("x"), …]
  time                :: Float64  [1990.0, 1990.08333333333, 1990.16666666667, 1990.25, 1990.33333333333, …]
  date                :: Dates.Date  [Dates.Da


### `mutate` — Add or Transform Columns

`mutate` creates new columns or transforms existing ones using `column_name => function` pairs.

In [7]:
# Add a log-transformed value column
tbl_transformed = mutate(trn, log_value  = ct -> log.(ct.value))

glimpse(tbl_transformed)

# Add multiple columns
tbl_enriched = mutate(trn,
      log_value  = ct -> log.(ct.value),
      year       = ct -> floor.(Int, ct.time),
      month_frac = ct -> ct.time .- floor.(ct.time)
  )

glimpse(tbl_enriched)

Table glimpse
  Rows: 141858
  Columns: 10
  data                :: String3  [String3("M3"), String3("M3"), String3("M3"), String3("M3"), String3("M3"), …]
  series              :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  period              :: String7  [String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]
  description         :: String  ["SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", …]
  value_type          :: String3  [String3("x"), String3("x"), String3("x"), String3("x"), String3("x"), …]
  time                :: Float64  [1990.0, 1990.08333333333, 1990.16666666667, 1990.25, 1990.33333333333, …]
  date                :: Dates.Date  [Dates.D

In [8]:
# across with explicit column list — applies log(x+1) to both :log_value and :value
glimpse(mutate(tbl_enriched, across([:log_value, :value], :log => x -> log.(x .+ 1))))

Table glimpse
  Rows: 141858
  Columns: 14
  data                :: String3  [String3("M3"), String3("M3"), String3("M3"), String3("M3"), String3("M3"), …]
  series              :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  period              :: String7  [String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]
  description         :: String  ["SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", …]
  value_type          :: String3  [String3("x"), String3("x"), String3("x"), String3("x"), String3("x"), …]
  time                :: Float64  [1990.0, 1990.08333333333, 1990.16666666667, 1990.25, 1990.33333333333, …]
  date                :: Dates.Date  [Dates.D


### `groupby` and `summarise` — Aggregation

`groupby` splits the data by one or more key columns, and `summarise` (or `summarize`) computes aggregate statistics for each group.

In [9]:
using Statistics
# Group training data by series
grouped = groupby(trn, :series)

# Summarise: compute statistics per series
series_stats = summarise(grouped,                                                                                                                                                                                                                      
      n_obs    = group -> length(group.value),
      mean_val = group -> mean(group.value),                                                                                                                                                                                                             
      sd_val   = group -> std(group.value),                                                                                                                                                                                                              
      min_val  = group -> minimum(group.value),
      max_val  = group -> maximum(group.value)
  )

glimpse(series_stats)

Table glimpse
  Rows: 1428
  Columns: 6
  series              :: String7  [String7("N1402"), String7("N1403"), String7("N1404"), String7("N1405"), String7("N1406"), …]
  n_obs               :: Int64  [50, 50, 50, 50, 50, …]
  mean_val            :: Float64  [3609.6, 1593.6, 3909.0, 2582.4, 5870.0, …]
  sd_val              :: Float64  [1950.3479124484013, 1422.9686703278796, 1811.2733709989213, 1916.8112977078385, 3198.7880357971803, …]
  min_val             :: Float64  [480.0, 120.0, 720.0, 180.0, 600.0, …]
  max_val             :: Float64  [9000.0, 7440.0, 8760.0, 7780.0, 12050.0, …]


In [10]:
# Group by domain type and compute counts
type_counts = summarise(
    groupby(trn, :type),
    n_series = group -> length(unique(group.series)),
    n_obs    = group -> length(group.value),
    avg_len  = group -> length(group.value) / length(unique(group.series))
)

glimpse(type_counts)

Table glimpse
  Rows: 6
  Columns: 4
  type                :: String15  [String15("DEMOGRAPHIC"), String15("FINANCE"), String15("INDUSTRY"), String15("MACRO"), String15("MICRO"), …]
  n_series            :: Int64  [111, 145, 334, 312, 474, …]
  n_obs               :: Int64  [11692, 15428, 40755, 35219, 35385, …]
  avg_len             :: Float64  [105.33333333333333, 106.4, 122.02095808383234, 112.88141025641026, 74.65189873417721, …]


> 📝 **Note**: `summarise` vs `summarize`
>
>
> Both spellings work identically. Use whichever you prefer.


## Reshaping Data

### `pivot_longer` — Wide to Long

`pivot_longer` converts wide-format data (one column per series) into long format (one row per observation). This is essential when data arrives as a spreadsheet with series in columns.

In [11]:
# Example: wide-format data
wide_data = (
    date     = ["2020-01", "2020-02", "2020-03"],
    series_A = [100.0, 105.0, 110.0],
    series_B = [200.0, 190.0, 195.0],
    series_C = [50.0, 55.0, 52.0]
)

# Pivot to long format
long_data = pivot_longer(wide_data;
    id_cols   = :date,
    names_to  = :series,
    values_to = :value
)

glimpse(long_data)
# (date = [...], series = [...], value = [...])
# Each row is now one observation: (date, series_name, value)

Table glimpse
  Rows: 9
  Columns: 3
  date                :: String  ["2020-01", "2020-01", "2020-01", "2020-02", "2020-02", …]
  series              :: String  ["series_A", "series_B", "series_C", "series_A", "series_B", …]
  value               :: Float64  [100.0, 200.0, 50.0, 105.0, 190.0, …]



### `pivot_wider` — Long to Wide

The inverse operation: spread a long table into wide format.

In [12]:
# Pivot back to wide format
wide_again = pivot_wider(long_data;
    id_cols    = :date,
    names_from = :series,
    values_from = :value
)

glimpse(wide_again)
# (date = [...], series_A = [...], series_B = [...], series_C = [...])

Table glimpse
  Rows: 3
  Columns: 4
  date                :: String  ["2020-01", "2020-02", "2020-03"]
  series_A            :: Float64  [100.0, 105.0, 110.0]
  series_B            :: Float64  [200.0, 190.0, 195.0]
  series_C            :: Float64  [50.0, 55.0, 52.0]



## Joins

TableOps provides the standard set of relational joins:

In [13]:
# Example tables
series_meta = (
    series = ["N0001", "N0002", "N0003"],
    domain = ["MICRO", "MACRO", "FINANCE"],
    country = ["US", "UK", "DE"]
)

series_values = (
    series = ["N0001", "N0001", "N0002", "N0002"],
    time   = [1990.0, 1991.0, 1990.0, 1991.0],
    value  = [100.0, 110.0, 200.0, 210.0]
)

# Inner join: only matching rows
result = inner_join(series_values, series_meta, by = :series)
glimpse(result)

# Left join: keep all rows from the left table
result = left_join(series_values, series_meta, by = :series)
glimpse(result)

# Full join: keep all rows from both tables
result = full_join(series_values, series_meta, by = :series)
glimpse(result)

# Semi join: filter left table to rows with matches in right
result = semi_join(series_values, series_meta, by = :series)
glimpse(result)

# Anti join: filter left table to rows WITHOUT matches in right
result = anti_join(series_values, series_meta, by = :series)
glimpse(result)

Table glimpse
  Rows: 4
  Columns: 5
  series              :: String  ["N0001", "N0001", "N0002", "N0002"]
  time                :: Float64  [1990.0, 1991.0, 1990.0, 1991.0]
  value               :: Float64  [100.0, 110.0, 200.0, 210.0]
  domain              :: String  ["MICRO", "MICRO", "MACRO", "MACRO"]
  country             :: String  ["US", "US", "UK", "UK"]
Table glimpse
  Rows: 4
  Columns: 5
  series              :: String  ["N0001", "N0001", "N0002", "N0002"]
  time                :: Float64  [1990.0, 1991.0, 1990.0, 1991.0]
  value               :: Float64  [100.0, 110.0, 200.0, 210.0]
  domain              :: Union{Missing, String}  ["MICRO", "MICRO", "MACRO", "MACRO"]
  country             :: Union{Missing, String}  ["US", "US", "UK", "UK"]
Table glimpse
  Rows: 5
  Columns: 5
  series              :: Union{Missing, String}  ["N0001", "N0001", "N0002", "N0002", "N0003"]
  time                :: Union{Missing, Float64}  [1990.0, 1991.0, 1990.0, 1991.0, missing]
  value       


## Additional Verbs

### `rename` — Rename Columns

In [14]:
renamed = rename(tbl_monthly, :y => :value, :series_id => :series)
glimpse(renamed)

Table glimpse
  Rows: 167562
  Columns: 9
  data                :: String3  [String3("M3"), String3("M3"), String3("M3"), String3("M3"), String3("M3"), …]
  series_id           :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  period              :: String7  [String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]
  description         :: String  ["SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", …]
  value_type          :: String3  [String3("x"), String3("x"), String3("x"), String3("x"), String3("x"), …]
  time                :: Float64  [1990.0, 1990.08333333333, 1990.16666666667, 1990.25, 1990.33333333333, …]
  date                :: Dates.Date  [Dates.Da


### `distinct` — Unique Rows

In [15]:
# Unique series identifiers
unique_series = distinct(select(trn, :series, :type))
glimpse(unique_series)

Table glimpse
  Rows: 1428
  Columns: 2
  series              :: String7  [String7("N1402"), String7("N1403"), String7("N1404"), String7("N1405"), String7("N1406"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]



### `bind_rows` — Stack Tables Vertically

In [16]:
# Combine multiple tables
all_data = bind_rows(trn, test)
glimpse(all_data)

Table glimpse
  Rows: 167562
  Columns: 9
  data                :: String3  [String3("M3"), String3("M3"), String3("M3"), String3("M3"), String3("M3"), …]
  series              :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  period              :: String7  [String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]
  description         :: String  ["SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", …]
  value_type          :: String3  [String3("x"), String3("x"), String3("x"), String3("x"), String3("x"), …]
  time                :: Float64  [1990.0, 1990.08333333333, 1990.16666666667, 1990.25, 1990.33333333333, …]
  date                :: Dates.Date  [Dates.Da


## Practical Workflow: Preparing M3 Data

Let's walk through a complete data preparation workflow for the M3 monthly dataset.

### Step 1: Load All Frequencies

In [18]:
using Durbyn
using Durbyn.TableOps
using CSV, Tables

data_dir = joinpath(@__DIR__, "..", "data")

tbl_monthly   = Tables.columntable(CSV.File(joinpath(data_dir, "M3_MONTHLY.csv")))
tbl_quarterly = Tables.columntable(CSV.File(joinpath(data_dir, "M3_QUARTERLY.csv")))
tbl_yearly    = Tables.columntable(CSV.File(joinpath(data_dir, "M3_YEARLY.csv")))

(data = String3["M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3"  …  "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3", "M3"], series = String7["N0001", "N0001", "N0001", "N0001", "N0001", "N0001", "N0001", "N0001", "N0001", "N0001"  …  "N0645", "N0645", "N0645", "N0645", "N0645", "N0645", "N0645", "N0645", "N0645", "N0645"], period = String7["YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY"  …  "YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY", "YEARLY"], type = String15["MICRO", "MICRO", "MICRO", "MICRO", "MICRO", "MICRO", "MICRO", "MICRO", "MICRO", "MICRO"  …  "OTHER", "OTHER", "OTHER", "OTHER", "OTHER", "OTHER", "OTHER", "OTHER", "OTHER", "OTHER"], description = ["SALES ( CODE= ABT)", "SALES ( CODE= ABT)", "SALES ( CODE= ABT)", "SALES ( CODE= ABT)", "SALES ( CODE= ABT)", "SALES ( CODE= ABT)", "SALES ( CODE= ABT)", "SALES ( CODE= ABT)", "SALES ( CODE= ABT)", "SALES ( CODE= ABT)"  …  


### Step 2: Explore the Data

In [19]:
# How many series per frequency?
for (name, tbl) in [("Monthly", tbl_monthly), ("Quarterly", tbl_quarterly), ("Yearly", tbl_yearly)]
    train = query(tbl, row -> row.value_type == "x")
    n_series = length(unique(trn.series))
    n_obs = length(trn.value)
    println("$name: $n_series series, $n_obs total observations")
end

Monthly: 1428 series, 141858 total observations
Quarterly: 1428 series, 141858 total observations
Yearly: 1428 series, 141858 total observations



### Step 3: Separate Train and Test

In [20]:
# Split monthly data
train_monthly = query(tbl_monthly, row -> row.value_type == "x")
test_monthly  = query(tbl_monthly, row -> row.value_type == "xx")

println("Train: $(length(train_monthly.value)) obs")
println("Test:  $(length(test_monthly.value)) obs")

glimpse(train_monthly)
glimpse(test_monthly)

Train: 141858 obs
Test:  25704 obs
Table glimpse
  Rows: 141858
  Columns: 9
  data                :: String3  [String3("M3"), String3("M3"), String3("M3"), String3("M3"), String3("M3"), …]
  series              :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  period              :: String7  [String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]
  description         :: String  ["SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", …]
  value_type          :: String3  [String3("x"), String3("x"), String3("x"), String3("x"), String3("x"), …]
  time                :: Float64  [1990.0, 1990.08333333333, 1990.16666666667, 1990.25, 1990.33333333333, …]
  date     


### Step 4: Compute Series-Level Statistics

In [21]:
# Group by series and compute descriptive statistics
series_summary = summarise(
    groupby(train_monthly, :series),
    type     = group -> first(group.type),
    n_obs    = group -> length(group.value),
    mean     = group -> round(mean(group.value), digits=2),
    std      = group -> round(std(group.value), digits=2),
    cv       = group -> round(std(group.value) / mean(group.value), digits=3),
    min_time = group -> minimum(group.time),
    max_time = group -> maximum(group.time)
)

glimpse(series_summary)

# Which types have the most series?
type_summary = summarise(
    groupby(series_summary, :type),
    count    = group -> length(group.series),
    avg_len  = group -> round(mean(group.n_obs), digits=1),
    avg_cv   = group -> round(mean(group.cv), digits=3)
)

glimpse(type_summary)

Table glimpse
  Rows: 1428
  Columns: 8
  series              :: String7  [String7("N1402"), String7("N1403"), String7("N1404"), String7("N1405"), String7("N1406"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]
  n_obs               :: Int64  [50, 50, 50, 50, 50, …]
  mean                :: Float64  [3609.6, 1593.6, 3909.0, 2582.4, 5870.0, …]
  std                 :: Float64  [1950.35, 1422.97, 1811.27, 1916.81, 3198.79, …]
  cv                  :: Float64  [0.54, 0.893, 0.463, 0.742, 0.545, …]
  min_time            :: Float64  [1990.0, 1990.0, 1990.0, 1990.0, 1990.0, …]
  max_time            :: Float64  [1994.08333333333, 1994.08333333333, 1994.08333333333, 1994.08333333333, 1994.08333333333, …]
Table glimpse
  Rows: 6
  Columns: 4
  type                :: String15  [String15("DEMOGRAPHIC"), String15("FINANCE"), String15("INDUSTRY"), String15("MACRO"), String15("MICRO"), …]
  count              


### Step 5: Extract a Single Series for Analysis

In [22]:
# Pick one series to work with
series_id = "N1402"

one_series = query(train_monthly, row -> row.series == series_id)
y = one_series.value  # This is the numeric vector Durbyn models expect

println("Series: $series_id")
println("Length: $(length(y))")
println("Range: $(minimum(y)) to $(maximum(y))")

Series: N1402
Length: 50
Range: 480.0 to 9000.0



### Step 6: Create PanelData for Multi-Series Modelling

In [ ]:
# PanelData wraps a table for grouped model fitting
# Select a subset of series for demonstration
demo_series = ["N1402", "N1403", "N1404", "N1405", "N1406"]

demo_data = query(train_monthly, row -> row.series in demo_series)
demo_data = select(demo_data, :series, :date, :value)

# Create PanelData
panel = PanelData(demo_data; groupby=:series, date=:date, m=12)
glimpse(panel)

panel

PanelData glimpse
  Groups: series
  Date: date
  Seasonal period m: 12
Table glimpse
  Rows: 250
  Columns: 3
  series              :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  date                :: Dates.Date  [Dates.Date("1990-01-01"), Dates.Date("1990-02-01"), Dates.Date("1990-03-01"), Dates.Date("1990-04-01"), Dates.Date("1990-05-01"), …]
  value               :: Float64  [2640.0, 2640.0, 2160.0, 4200.0, 3360.0, …]


PanelData
─────────
  Groups: series
  Date column: date
  Seasonal period: 12
  Data: 250 rows × 3 columns



## `glimpse` — Quick Data Inspection

The `glimpse` function works on tables, PanelData, fitted models, and forecasts:

In [24]:
# On a table
glimpse(train_monthly)

# On PanelData
glimpse(panel)

# On fitted models (after fitting)
# glimpse(fitted_models)

# On forecasts
# glimpse(fc)

Table glimpse
  Rows: 141858
  Columns: 9
  data                :: String3  [String3("M3"), String3("M3"), String3("M3"), String3("M3"), String3("M3"), …]
  series              :: String7  [String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), String7("N1402"), …]
  period              :: String7  [String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), String7("MONTHLY"), …]
  type                :: String15  [String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), String15("MICRO"), …]
  description         :: String  ["SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", "SHIPMENTS (Code TD-30EXP)", …]
  value_type          :: String3  [String3("x"), String3("x"), String3("x"), String3("x"), String3("x"), …]
  time                :: Float64  [1990.0, 1990.08333333333, 1990.16666666667, 1990.25, 1990.33333333333, …]
  date                :: Dates.Date  [Dates.Da